Imports

In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    explained_variance_score,
    r2_score,
)
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

Read data

In [20]:
maindf = pd.read_csv('./BTC_USD.csv')
print("Total number of days present in the dataset: ", maindf.shape[0])
print("Total number of fields present in the dataset: ", maindf.shape[1])


# Check for null and NA values

print('Null Values: ', maindf.isnull().values.sum())
print("NA values:", maindf.isnull().values.any())

#Data shape + look
maindf.shape

maindf.describe()

Total number of days present in the dataset:  3621
Total number of fields present in the dataset:  7
Null Values:  0
NA values: False


,Open,High,Low,Close,Adj Close,Volume
count,3621.000000,3621.000000,3621.000000,3621.000000,3621.000000,3.621000e+03
mean,17430.522947,17827.991156,17004.375770,17444.936800,17444.936800,1.751433e+10
std,19305.417630,19740.040312,18824.610395,19312.520889,19312.520889,1.921478e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,1116.319946,1151.739990,1077.250000,1117.439941,1117.439941,2.721670e+08
50%,9048.460938,9217.835938,8811.366211,9045.390625,9045.390625,1.367521e+10
75%,28522.097656,29159.902344,28174.251953,28627.574219,28627.574219,2.828269e+10
max,73079.375000,73750.070313,71334.093750,73083.500000,73083.500000,3.509679e+11
